# Parsing tutorial

## Preamble

The goal of this notebook is to walk you through one of many possible ways of 
parsing a GFF3 file.

A few notes about this notebook:

- You can run the **code cells**. I recommend running them one by one using 
`shift` + `enter`.
- You would typically use `sys`'s `argv` module to input the program's 
arguments, but here they will be hardcoded due to the nature of the notebook.
- This also means that the code won't have the typical structure, but remember
that your code should look like this:

```python
#!/usr/bin/env python3

"""
Describe what your program does and how to run it. Also put your name here
"""

from sys import argv

def main():
    # here you would put calls to each of your functions
    pass

if __name__ == "__main__":
    main()
```

# A GFF3 parser

A GFF3 (Generic Feature Format Version 3) file contains annotations for a 
genomic sequence. These include genes, mRNA, coding sequence (CDS), exons and
other features.

The file that we will work on also contains the DNA sequence itself (though this
is valid, the DNA sequence is found in a separate fasta file). This data is a 
subset from an assembly of [*Catharanthus roseus*](https://www.ncbi.nlm.nih.gov/labs/data-hub/genome/GCA_000949345.1/), 
a plant from which the anti-cancer drug [vinblastine](https://en.wikipedia.org/wiki/Vinblastine) 
and [vincristine](https://en.wikipedia.org/wiki/Vincristine) are extracted.

The goals of this exercise are:

* **Create a fasta file with the translations of all the coding sequences**
* **Calculate which percentage of the total sequence is coding**

We will go through the first steps of parsing the file so that you end up with
a data structure that will allow you to have the required output.

## The input data

**STOP** ✋ First, let's take a look at the file. You can download it 
[here](https://drive.google.com/file/d/1q_vny3l-3qjhwjO5_9_PLHeU3FGMOFvW/view?usp=sharing).

Scroll down to the end. What do you see? How many different sections does it 
have? Is there a clear way of separating each section/subsection?

### The GFF3 structure: Part 1

Here, we start with a header that has the version of the format used, followed
by a tab-delimited section with the actual annotations:

```
##gff-version 3
cro_scaffold_3066516	maker	gene	31108	33451	.	+	.	ID=CRO_004871;Name=CRO_004871
cro_scaffold_3066516	maker	mRNA	31108	33451	.	+	.	ID=CRO_T004871;Name=CRO_T004871;Parent=CRO_004871
cro_scaffold_3066516	maker	exon	31108	31466	.	+	.	Parent=CRO_T004871
cro_scaffold_3066516	maker	exon	31563	31655	.	+	.	Parent=CRO_T004871
cro_scaffold_3066516	maker	exon	31779	31991	.	+	.	Parent=CRO_T004871
cro_scaffold_3066516	maker	exon	32088	32267	.	+	.	Parent=CRO_T004871
cro_scaffold_3066516	maker	exon	32376	33451	.	+	.	Parent=CRO_T004871
cro_scaffold_3066516	maker	five_prime_UTR	31108	31169	.	+	.	Parent=CRO_T004871
cro_scaffold_3066516	maker	CDS	31170	31466	.	+	0	Parent=CRO_T004871
cro_scaffold_3066516	maker	CDS	31563	31655	.	+	0	Parent=CRO_T004871
cro_scaffold_3066516	maker	CDS	31779	31991	.	+	0	Parent=CRO_T004871
cro_scaffold_3066516	maker	CDS	32088	32267	.	+	0	Parent=CRO_T004871
cro_scaffold_3066516	maker	CDS	32376	33335	.	+	0	Parent=CRO_T004871
cro_scaffold_3066516	maker	three_prime_UTR	33336	33451	.	+	.	Parent=CRO_T004871
...
```

### The GFF3 structure: Part 2

At the bottom of the file, we have what looks like a normal fasta file:
```
...
cro_scaffold_3069583	maker	exon	103062	103433	.	-	.	Parent=CRO_T020549
cro_scaffold_3069583	maker	CDS	103062	103433	.	-	0	Parent=CRO_T020549
##FASTA
>cro_scaffold_3069583
GGTCCAAAATCCAATATTGTGAATAATTGTTTTCGACACGCAACATAAAATCCTTATTAAACAAACAAAAGTTGATAAAA
AATAATATATTTTATTAGGATGATCTAGGAAAATAATTGAGTATAGTTCCTAGGATGGTAGGAAATTAATAATAATACCC
GAAAATAAATGATATTAACAATAACATAACTATAAATAATAAACATGTCATATCACCGTAGCTGTCATGAAAATATTTTA
CCATCAAAATTTCTCAAAAATAAAATAAGAAATAATTAAACTATATACTTAAGATTATGATGTAAATAGTTTTTCATTAT
ATATATATCAAATATCATGATTGGCAAACCACTTTTTTTTTTTTTTTTTTTTTAAGTCTCCACTTGGTTATGGTACAAAA
...
```

Importantly, we see a clear separator for the two main sections: the string
'`##FASTA`'.

## Part 2: Planning

Now that we have taken a look at the file we'll work with, we have to decide
which data structure we will use to store our data. Thinking on this now will
guide the building of the code that follows.

### The sequence data

The bottom part is easier to think about. As we will need to retrieve the DNA
sequences to translate them, and these all have a contig label (and moreover, 
this label is unique!), the easiest data structure here is a dictionary:

```python
fasta['contig label'] -> "str"
```

### The annotation data

This part is clearly more complicated. Let's start with what kind of data we 
need in order to answer the exercise. 

The first task consists in producing the translations of all coding sequences.
This means we need to take the right contig. Conveniently, this is indicated
in the **first column**, where the contig labels (must!) match the ones in the 
fasta section. 

The type of feature that we need is the coding sequence, or **CDS**. As a 
reminder, see the full gene structure from this image from [Wikipedia](https://en.wikipedia.org/wiki/Gene_structure#Eukaryotes)
(we need the subsequences corresponding to the red parts):

![eukaryotic gene structure](Gene_structure_eukaryote_2_annotated.png)

The type of feature is annotated in the **third column**. For any given 
transcript, there may be multiple CDS regions indicated, which represent the 
coding regions within multiple exons.

We also need, of course, the actual coordinates, which are stored in the **4th
and 5th columns** (are they 0- or 1-based indexed? see [here](https://github.com/The-Sequence-Ontology/Specifications/blob/master/gff3.md#description-of-the-format) for the answer)

Another important piece of information is the strand that the transcript is in,
which is stored in the **7th column**.

If we only filtered all the "CDS"-containing lines, it would be impossible to
know which one(s) belong to the same transcript. For this reason, we also need
the information in the **9th column**, which will contains the transcript ID
that we need to group the coordinates.

To wrap this up, see this annotated figure of the gff file:

![annotation section](annotation_section.png)

### Data structures

How to store this data? The coordinates are the easiest as they won't be changed
and a tuple containing both of them as integers is an obvious choice:

```python
coordinate -> tuple([start:int, end:int])
```

For the rest, there are many options. One of them is having a dictionary linking
contigs to transcripts:

```python
annotations['contig label'] -> list('transcript label 1', 'transcript label 2', ...)
```

but then we'd need another structure to link the transcript labels to their 
coordinates.

A general advice is to always use as few structures as possible. Moreover, it
would be convenient to see the output fasta writer function as:

```python
# pseudocode:
with open("translation.fasta", "w") as res:
    for every contig label:
        # obtain contig sequence here
        for every transcript label:
            translation = get_translation(contig_sequence, coordinates linked to transcript)
            # write transcript label + translation to `res`
```

Here, data would have a clear hierarchy:

```
hierarchy: contig label -> transcript label -> coordinates -> start,stop
```

This lays out a nested data structure:
```python
# 'CDS' is a tuple with the coding region coordinates within one exon as integers
CDS -> tuple([start:int, end:int])

# 'coordinates' is a list of CDSs belonging to a given transcript
coordinates -> [CDS_1, CDS_2, ...]

# 'transcripts' is a dictionary linking transcript labels to lists of CDSs 
# belonging to that transcript
transcripts['transcipt label i'] -> coordinates_i

# the main data structure is also a dictionary, holding all of the transcripts 
# with their CDSs for a given contig. (In this file, there are only two contigs)
main_dictionary['contig label j'] -> transcripts_contig_j
```

There's one thing missing: the strand information. We do need this information
when we translate the sequences, to know if we need to first do the reverse
complement. Here, the easiest would be to keep this information elsewhere, in
its own dictionary:

```python
strand['transcript label'] -> str
```

Now we are ready to begin our program... although we haven't discussed anything
related to the second task: calculating the percentage of sequence that is 
coding. Think: Can we do this with the current data structures?

With all of these in mind, here is the skeleton of our parsing function:

In [ ]:
# In the definition of the function, you can indicate the data types of the 
#   arguments and returns. This is not enforced by the Python interpreter but
#   can be used by other tools.
def gff_parser(filename: str) -> tuple([dict, dict]):
    """Parses a GFF3 file
    
    Parameters:
    ---
    filename: str
        Name of the GFF3 file

    Returns:
    ---
    A tuple with two elements:
        dict: key=contig label, value=a DNA string
        dict: key=transcript label, value=a list of tuples representing the
            coordinates of each CDS
    """

    fasta = {}
    annotation = {}
    strand = {}

    # [PARSING ALGORITHM HERE]
                
    return fasta, annotation, strand

fasta_data, annotation_data, strand_data = gff_parser("Catharanthus.gff3")

# Part three: the parser

The main logic step is to divide the processing of the fasta section and the 
annotation section. For this I will use a function when I reach the '`##FASTA`'
keyword. The function currently looks like this:

In [ ]:
def gff_parser(filename: str) -> tuple([dict, dict]):
    """Parses a GFF3 file
    
    Parameters:
    ---
    filename: str
        Name of the GFF3 file

    Returns:
    ---
    A tuple with two elements:
        dict: key=contig label, value=a DNA string
        dict: key=transcript label, value=a list of tuples representing the
            coordinates of each CDS
    """

    fasta = {}
    annotation = {}
    strand = {}

    # parsing algorithm
    with open(filename) as gff:
        for line in gff:
            if line.startswith("##FASTA"):
                # [LAUNCH FUNCTION TO PARSE FASTA SECTION]
                continue
            else:
                # [IMPLEMENT LOGIC FOR PARSING ANNOTATION SECTION]
                continue
                
    return fasta, annotation, strand

Another option is to implement the fasta-parsing logic directly in `gff_parser`.
For that we could define a flag that would be activated when we reach the
separator:

```python
in_fasta_section = False

with open(filename) as gff:
    for line in gff:
        if in_fasta_section:
            # [IMPLEMENT LOGIC FOR PARSING FASTA SECTION DIRECTLY]
            continue
        else:
            # Activate the flag when separator is found. After that we
            # won't enter this section again
            if line.startswith("##FASTA"):
                in_fasta_section = True
                continue

            # [IMPLEMENT LOGIC FOR PARSING ANNOTATION SECTION]
```

For convenience (and clarity), I will use a separate function that takes the 
[file object](https://docs.python.org/3.11/tutorial/inputoutput.html#reading-and-writing-files) 
returned by `open()`. I'm using the fact that the file is still opened and we
can continue looping lines from the *current position*. Attention! This means 
that we should call that function immediately after finding the section 
separator, before the next line in the file is read!:

In [ ]:
def gff_parser(filename: str) -> tuple([dict, dict]):
    """Parses a GFF3 file
    
    Parameters:
    ---
    filename: str
        Name of the GFF3 file

    Returns:
    ---
    A tuple with two elements:
        dict: key=contig label, value=a DNA string
        dict: key=transcript label, value=a list of tuples representing the
            coordinates of each CDS
    """

    fasta = {}
    annotation = {}
    strand = {}

    # parsing algorithm
    with open(filename) as gff:
        for line in gff:
            if line.startswith("##FASTA"):
                # Launch a function that parses the fasta section
                fasta = parse_fasta(gff)
            else:
                # [IMPLEMENT LOGIC FOR PARSING ANNOTATION SECTION]
                continue
                
    return fasta, annotation, strand


⚠️ Warning! This assumes that we won't find another annotation section. Always 
check your input file and, if possible, read documentation about the format!

As we said, we will store the DNA sequence in a dictionary. A possible 
optimization here would be to only include in the dictionary the sequences that
we actually have from the annotation section.

Another thing to consider is how big are the input files. For small files, 
taking the complete data into a dictionary is not a big issue, but for 
assemblies/projects with many/large records (see for example [this newer 
assembly](https://www.ncbi.nlm.nih.gov/labs/data-hub/genome/GCA_024505715.1/)
of the same organism!), it would be a good idea to refactor this code to only 
yield the sequences we need, when we need them.

### The fasta parser

You've likely made your own fasta parser by now, so I won't spend a lot of time
here. The basic idea is similar to the gff parser: there are two kinds of 
sections: header lines and sequence lines.

In [ ]:
def parse_fasta(fasta_file_object) -> dict:
    """Reads fasta data from a file object and returns data as a dictionary
    
    Parameters:
    ---
    fasta_file_object: file object
        A(n already opened) file object

    Return:
    ---
    dict
        A dictionary with key=sequence label; value=sequence
    """

    fasta = {}
    
    for line in fasta_file_object:
        clean_line = line.strip()

        # Skip empty lines
        if clean_line == "":
            continue

        if clean_line[0] == '>':
            # [DO SOMETHING WITH A HEADER LINE]
            continue
        else:
            # [DO SOMETHING WITH A SEQUENCE LINE]
            continue

    return fasta

fasta_data, annotation_data, strand_data = gff_parser("Catharanthus.gff3")

for label, seq in fasta_data.items():
    print(label, len(seq))

**STOP** ✋ Can you finish the fasta parser yourself?

These are the values you should have if you run the previous cells:

```
cro_scaffold_3069583 250200
cro_scaffold_3066516 209353
```

```





```

Ok, let's continue!

If we have a sequence line: simply add the cleaned line to the sequence list.

If we have a label line: we need to convert the subsequence list into a proper
string, store it in the final dictionary using the current label, get the new
label and reset the subsequence list:

In [ ]:
def parse_fasta(fasta_file_object) -> dict:
    """Reads fasta data from a file object and returns data as a dictionary
    
    Parameters:
    ---
    fasta_file_object: file object
        A(n already opened) file object

    Return:
    ---
    dict
        A dictionary with key=sequence label; value=sequence
    """

    fasta = {}
    label = ""
    sequence = []
    for line in fasta_file_object:
        clean_line = line.strip()

        # Skip empty lines
        if clean_line == "":
            continue

        if clean_line[0] == '>':
            # Store current data; Prepare for the next record
            fasta[label] = "".join(sequence)

            label = clean_line[1:]
            sequence = []
        else:
            # Keep adding sequence data
            sequence.append(clean_line)

    return fasta

fasta_data, annotation_data, strand_data = gff_parser("Catharanthus.gff3")

for label, seq in fasta_data.items():
    print(label, len(seq))

**STOP** ✋ Can you figure out what went wrong?

As a reminder, these are the values you should have if you run the previous 
cells:

```
cro_scaffold_3069583 250200
cro_scaffold_3066516 209353
```

```






```

There are other ways of concatenating the subsequences, but the one I used above
is fast (see a note about this in the "On Parsing / Reading the data / Tips and
tricks" module in Brightspace)

This works for all the fasta records in the middle of the file, but special 
considerations need to be taken for the first record (we find a '>' character, 
but we don't have a previous subsequence list) and the last one (we run out of
lines to loop on and we won't be getting another '>' character to indicate we 
have to store the last record). The final fasta-parsing function would look like
this (note that here we could do some extra processing, like making sure all 
bases are upper case):

In [ ]:
def parse_fasta(fasta_file_object) -> dict:
    """Reads fasta data from a file object and returns data as a dictionary
    
    Parameters:
    ---
    fasta_file_object: file object
        A(n already opened) file object

    Return:
    ---
    dict
        A dictionary with key=sequence label; value=sequence
    """

    fasta = {}
    label = ""
    sequence = []
    for line in fasta_file_object:
        clean_line = line.strip()

        # Skip empty lines
        if clean_line == "":
            continue

        if clean_line[0] == '>':
            # We found a header

            # This evaluates to False if sequence is an empty list (which will
            # happen in the very first line)
            if sequence:
                fasta[label] = "".join(sequence)

            label = clean_line[1:]
            sequence = []
        else:
            # Keep adding sequence data
            sequence.append(clean_line)

    # Handle the last record
    fasta[label] = "".join(sequence)

    return fasta

fasta_data, annotation_data, strand_data = gff_parser("Catharanthus.gff3")

for label, seq in fasta_data.items():
    print(label, len(seq))

```
cro_scaffold_3069583 250200
cro_scaffold_3066516 209353
```

Yes, we got it!

### The annotations parser

Now it's time to continue working on parsing the annotation section. The header
of this section contains lines starting with the '#' symbol that we can safely
ignore (a single '#' represents a comment, but lines starting with double '##' 
can also contain specific metadata and are called "pragmas"). 

The rest of the lines are easy to parse as they are a regular tab-delimited 
table. We already discussed which columns we need, so let's get them:

In [ ]:
def gff_parser(filename: str) -> tuple([dict, dict]):
    """Parses a GFF3 file
    
    Parameters:
    ---
    filename: str
        Name of the GFF3 file

    Returns:
    ---
    A tuple with two elements:
        dict: key=contig label, value=a DNA string
        dict: key=transcript label, value=a list of tuples representing the
            coordinates of each CDS
    """

    fasta = {}
    annotation = {}
    strand = {}

    # parsing algorithm
    with open(filename) as gff:
        for line in gff:
            if line.startswith("##FASTA"):
                # Launch a function that parses the fasta section
                fasta = parse_fasta(gff)
            else:
                # Ignore comments and "pragmas"
                if line[0] == '#':
                    continue

                # Get all data from line
                columns = line.strip().split("\t")
                contig_label = columns[0]
                feature_type = columns[2]
                start = columns[3]
                end = columns[4]
                strand_symbol = columns[6]
                # the transcript ID needs more cleaning
                transcript_label = columns[8].partition("Parent=")[2].split(";")[0]

                # ignore all features except CDS
                if feature_type != "CDS":
                    continue
                
                
    return fasta, annotation, strand

fasta_data, annotation_data, strand_data = gff_parser("Catharanthus.gff3")


**STOP** ✋ We already discussed which data structures to use. Can you modify 
the above function to populate `annotation_data`? Try it out and run the next
cell, which should print `85393	85925`:

In [ ]:
try:
    print(annotation_data['cro_scaffold_3069583']['CRO_T020528'][1])
except KeyError:
    print("Data structure incomplete...")
except IndexError:
    print("Data structure incomplete...")

```






```

Ok, let's populate the data structure:

In [ ]:
def gff_parser(filename: str) -> tuple([dict, dict]):
    """Parses a GFF3 file
    
    Parameters:
    ---
    filename: str
        Name of the GFF3 file

    Returns:
    ---
    A tuple with two elements:
        dict: key=contig label, value=a DNA string
        dict: key=transcript label, value=a list of tuples representing the
            coordinates of each CDS
    """

    fasta = {}
    annotation = {}
    strand = {}

    # parsing algorithm
    with open(filename) as gff:
        for line in gff:
            if line.startswith("##FASTA"):
                # Launch a function that parses the fasta section
                fasta = parse_fasta(gff)
            else:
                # Ignore comments and "pragmas"
                if line[0] == '#':
                    continue

                # Get all data from line
                columns = line.strip().split("\t")
                contig_label = columns[0]
                feature_type = columns[2]
                start = int(columns[3]) - 1 # Convert to 0-based index!
                end = int(columns[4]) - 1
                strand_symbol = columns[6]
                # the transcript ID needs more cleaning
                transcript_label = columns[8].partition("Parent=")[2].split(";")[0]

                # ignore all features except CDS
                if feature_type != "CDS":
                    continue

                # Create entry for contig_label if needed
                if contig_label not in annotation:
                    annotation[contig_label] = {}

                # Initialize list of coordinates if needed
                if transcript_label not in annotation[contig_label]:
                    annotation[contig_label][transcript_label] = []

                # Append new tuple with coordinates
                CDS = tuple([start, end])
                annotation[contig_label][transcript_label].append(CDS)
                
                
    return fasta, annotation, strand

fasta_data, annotation_data, strand_data = gff_parser("Catharanthus.gff3")

try:
    print(annotation_data['cro_scaffold_3069583']['CRO_T020528'][1])
except KeyError:
    print("Data structure incomplete...")
except IndexError:
    print("Data structure incomplete...")

Alright! Here we finished with parsing the .gff file.

**STOP** ✋ Can you complete the exercise? Here are the functions that you
need to implement:

```python
def reverse_complement(seq:str) -> str:
    """Takes a DNA sequence and returns the reverse complement

    Parameters:
    ---
    seq: str
        A DNA sequence

    Returns:
    ---
        str
        The reverse complement of the original sequence
    """

    return


def translate(seq:str, strand_symbol:str) -> str:
    """Takes a DNA sequence and returns the translation

    Parameters:
    ---
    seq: str
        A DNA sequence
    strand_symbol: str

    Returns:
    ---
        str
        A protein sequence
    """

    return


def assemble_sequence(contig_sequence:str, CDSlist:list) -> str:
    """Assembles a complete coding sequence from a list of coordinates

    Parameters:
    ---
    contig_sequence: str
        The complete contig DNA sequence
    CDSlist: list
        A list of tuples representing start/end coordinates

    Returns:
    ---
        str
        A DNA sub-sequence
    """

    return


def percentage_coding(fasta_data:dict, annotation_data:dict) -> float:
    """Calculates which percentage from the total sequence length is coding

    Parameters:
    ---
    fasta_data: dict
        Keys=contig label, values=a DNA string
    annotation_data: dict
        Keys=contig label, values=a dictionary (with keys=transcript label, 
            values=list of tuples) 

    Returns:
    ---
        float
        The percentage of the total sequence length that is coding
    """

    return
```